In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv01 = nn.Sequential(         
            nn.Conv2d(64, 64*2, 1, 1, padding = "same"),
            nn.BatchNorm2d(64*2),
            nn.ReLU(),
            nn.Conv2d(64*2, 64*2, 3, 1, padding = "same"),
            nn.BatchNorm2d(64*2),
            nn.ReLU(),
            nn.Conv2d(64*2, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv02 = nn.Sequential(         
            nn.Conv2d(32, 32*8, 1, 1, padding = "same"),
            nn.BatchNorm2d(32*8),
            nn.ReLU(),
            nn.Conv2d(32*8, 32*8, 3, 1, padding = "same"),
            nn.BatchNorm2d(32*8),
            nn.ReLU(),
            nn.Conv2d(32*8, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        identity = x
        x = self.conv01(x)
        x = identity + x
        x = self.conv3(x)
        identity = x
        x = self.conv02(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv01): Sequential(
    (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv02): Sequential(
    (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(256, 32, kernel_size

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0375
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 2.9705
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.8879
7.609988212585449
epoch: 01 | Accuracy: 7.61 Loss: 81.5157

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.6013
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.6370
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.4661
18.66825294494629
epoch: 02 | Accuracy: 18.67 Loss: 72.0530

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 2.3518
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 2.2969
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 2.1861
32.104637145996094
epoch: 03 | Accuracy: 32.10 Loss: 59.6652

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.8203
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.7546
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.8482
45.422115325927734
epoch: 04 | Accuracy: 45.42 Loss: 48.1243

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 1.3414
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.9755
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.6072
59.453033447265625
epoch: 05 | Accuracy: 59.45 Loss: 35.6526

Epoch: [6/30]	|	Step: [

In [7]:
print(Accy_list)

[7.609988212585449, 18.66825294494629, 32.104637145996094, 45.422115325927734, 59.453033447265625, 72.88941955566406, 79.78597259521484, 86.92033386230469, 90.72532653808594, 95.4815673828125, 95.00594329833984, 94.53031921386719, 97.50297546386719, 99.28656005859375, 97.74078369140625, 99.52437591552734, 99.88109588623047, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[81.51568245887756, 72.05304980278015, 59.6651793718338, 48.12426173686981, 35.652554750442505, 24.226608216762543, 17.397880345582962, 12.25541128218174, 9.175254940986633, 5.847226545214653, 5.324641980230808, 5.36321921646595, 3.1554959267377853, 1.806590186432004, 2.8929367512464523, 1.1497807428240776, 0.4511949857696891, 0.23995024524629116, 0.18235831893980503, 0.16356390505097806, 0.13096521073020995, 0.09653931856155396, 0.08318158891052008, 0.0953848430654034, 0.08412047429010272, 0.08015940128825605, 0.05764794326387346, 0.053141166688874364, 0.04751390125602484, 0.04266264144098386]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 88.94 %, Loss: 12.5469 
